# Using the SSC Assistant API

This will demonstrate how to use the SSC Assistant API using your given API token and a user Access token (oauth2).

In [31]:
import os

import jwt
import msal
import requests
from dotenv import load_dotenv

load_dotenv()

api_endpoint = os.getenv("API_ENDPOINT")
api_scope = os.getenv("API_SCOPE")

print(api_scope)

app_id = os.getenv("APP_ID")
app_secret = os.getenv("APP_SECRET")
tenant_id = os.getenv("TENANT_ID")

app = msal.ConfidentialClientApplication(
    app_id, authority=f"https://login.microsoftonline.com/{tenant_id}",
    client_credential=app_secret)

result = None
result = app.acquire_token_silent(scopes=[api_scope], account=None)

if not result:
    print("No suitable token exists in cache. Let's get a new one from AAD.")
    result = app.acquire_token_for_client(scopes=[api_scope])

if result and "access_token" in result:
    access_token = result['access_token']
    decoded_token = jwt.decode(access_token, options={"verify_signature": False})
    print(decoded_token)

api://5e945d23-48d9-4929-b1a1-93a9ca58f8aa/.default
No suitable token exists in cache. Let's get a new one from AAD.
{'aud': 'api://5e945d23-48d9-4929-b1a1-93a9ca58f8aa', 'iss': 'https://sts.windows.net/d05bc194-94bf-4ad6-ae2e-1db0f2e38f5e/', 'iat': 1737049840, 'nbf': 1737049840, 'exp': 1737053740, 'aio': 'k2RgYNAoXJTFUHbDeEZa6aPHPmu/AAA=', 'appid': 'fa97a723-f604-438b-8bd6-06543065f6a9', 'appidacr': '1', 'idp': 'https://sts.windows.net/d05bc194-94bf-4ad6-ae2e-1db0f2e38f5e/', 'oid': '5605426c-1531-4da5-b6b6-2413d57a1446', 'rh': '1.ASkAlMFb0L-U1kquLh2w8uOPXiNdlF7ZSClJsaGTqcpY-Kr4AAApAA.', 'sub': '5605426c-1531-4da5-b6b6-2413d57a1446', 'tid': 'd05bc194-94bf-4ad6-ae2e-1db0f2e38f5e', 'uti': 'FA-DuKdNM0-oTz1ZgzGmAA', 'ver': '1.0'}


## Security

API Is secured and need two different token in order to access it, first an API token (see below) and a user access token (provided by microsoft idp)

Once we have the access_token we simply make a simple call to the SSC Assistant API to ask a one-of question.

### API Token

To do so we need to ensure we have a valid user (above) and a valid token to send in the `X-API-Key` header as part of the `POST` request.

Such token can be crafted for testing as bellow:

**NOTE:** If you are not part of the dev team and you require a real token please contact the SSC Assistant team.

In [32]:
import jwt
api_token = jwt.encode({'roles': ['mysscplus',]}, 'secret', algorithm='HS256')

question = {
  "query": "Who is the president of SSC?",
}

response = requests.post(str(api_endpoint) + "/api/1.0/mysscplus/suggest", headers={'Authorization': f'Bearer {access_token}', 'X-API-Key': api_token}, json=question)

if response.status_code == 200:
    print(response.json())
else:
    print("Error: ", response.text)

{'completion_tokens': 39, 'message': {'attachments': None, 'content': 'The current President of Shared Services Canada (SSC) is Sony Perron [doc6].', 'context': {'citations': [{'chunk_id': None, 'content': 'Goodbye, SSC Dear colleagues, Well, my last day has arrived. Not just my last day as President of SSC, but also my last day as a public servant.\u202fI want to thank everyone here at SSC for the kind words and congratulations.\u202fYour messages and your support mean the world to me. I feel incredibly privileged and proud of my 35 years as a public servant.\u202fI believe public service matters – it changes lives, and it is part of what makes Canada such a great country! My last three years with you here at SSC have been very special, and I definitely feel like I am leaving on a high note! As I said at my rather public exit interview, the problems of 10 years ago are not the problems we face today, and the challenges of tomorrow will be different from today. SSC has grown so much ov